In [1]:

n =100000
# Filter and format the proxies
filtered_proxies = []

with open('../PROXY-List/http.txt', 'r') as file:
    lines = file.readlines()

for line in lines[:n]:
    ip, port = line.strip().split(':')
    filtered_proxies.append(f'http://{ip}:{port}')

with open('../PROXY-List/socks4.txt', 'r') as file:
    lines = file.readlines()

for line in lines[:n]:
    ip, port = line.strip().split(':')
    filtered_proxies.append(f'socks4://{ip}:{port}')


# Read the contents of http.txt
with open('../PROXY-List/socks5.txt', 'r') as file:
    lines = file.readlines()

for line in lines[:n]:
    ip, port = line.strip().split(':')
    filtered_proxies.append(f'socks5://{ip}:{port}')

# Write the filtered proxies to proxy-list.txt
with open('proxy-list.txt', 'w') as file:
    for proxy in filtered_proxies:
        file.write(proxy + '\n')

print("Filtered proxies have been written to proxy-list.txt")


Filtered proxies have been written to proxy-list.txt


In [2]:
from dotenv import load_dotenv
import pandas as pd
import os
import ipaddress
import json
import requests
load_dotenv()


True

In [2]:
# import argparse
# import asyncio
# import random
# import signal
# import ssl
# import json
# import time
# import uuid
# from loguru import logger
# import websockets
# from websockets_proxy import Proxy, proxy_connect
# import os
# from dotenv import load_dotenv

# load_dotenv()

# http_proxy = 'socks5://167.88.175.12:34567'
# WEBSOCKET_URL = "wss://nw.nodepay.ai:4576/websocket"
# SERVER_HOSTNAME = "nw.nodepay.ai"
# browser_id = str(uuid.uuid3(uuid.NAMESPACE_DNS, http_proxy))
# logger.info(f"Browser ID: {browser_id}")

# retries = 0


# proxy = Proxy.from_url(http_proxy)
# custom_headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
# }
# ssl_context = ssl.create_default_context()
# ssl_context.check_hostname = False
# ssl_context.verify_mode = ssl.CERT_NONE
# async with proxy_connect(WEBSOCKET_URL, proxy=proxy, ssl=ssl_context, server_hostname=SERVER_HOSTNAME,
#                             extra_headers=custom_headers) as websocket:
#     logger.info("Connected to WebSocket")
#     retries = 0
#     async for message in websocket:
#         data = json.loads(message)
#         print(data)
#         if (data["action"] == "AUTH") or (data["action"] == "PONG") or (data["action"] == "PING") :
#                 print(data)
#                 await websocket.close()
    
#     await websocket.close()

In [3]:
# Function to check if IP is within the same /24 subnet
def is_same_subnet(ip1, ip2):
    ip1_subnet = ipaddress.ip_network(ip1 + '/24', strict=False)
    return ipaddress.ip_address(ip2) in ip1_subnet

# Function to read proxy list from a file
def read_proxy_list(filename):
    with open(filename, 'r') as file:
        return [line.strip() for line in file.readlines()]

In [17]:
max_page = 500
once_per_request = 100
API_KEY = os.getenv("API_KEY")
    # Define the CSV file name
payload = {}
headers = {
'Authorization': str(API_KEY)
}

df = pd.DataFrame([])

for i in range(max_page):
    url = "https://api.getgrass.io/devices?input=%7B%22limit%22:{},%22offset%22:{}%7D".format(once_per_request,i * once_per_request)
    response = requests.request("GET", url, headers=headers, data=payload, verify=False)
    json_data = json.loads(response.text)
    # Extract the relevant data
    devices_data = json_data['result']['data']['data']
    # Create a DataFrame
    devices_data = pd.DataFrame(devices_data)
    df = pd.concat([df,devices_data], axis= 0)



/home/aston/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.getgrass.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/aston/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.getgrass.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/aston/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.getgrass.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/aston/.venv/lib/python3.11/site-packages

In [5]:

bad_ip_list = []
for indx, row in df.iterrows():
    ipAddress = row['ipAddress']
    ipScore = row['ipScore']
    totalUptime = row['totalUptime']
    if (totalUptime > 600) and (ipScore == 0):
        bad_ip_list.append(ipAddress)

In [6]:
# Function to filter proxy IPs that are not in the same subnet as any bad IPs
def filter_proxies(proxy_list, bad_ip_list):
    filtered_proxies = []
    for proxy in proxy_list:
        proxy_ip = proxy.split('//')[1].split(':')[0]  # Extract IP address from URL
        if not any(is_same_subnet(bad_ip, proxy_ip) for bad_ip in bad_ip_list):
            filtered_proxies.append(proxy)
    return filtered_proxies

In [7]:
# Write the bad IPs to bad_ip.txt
with open('bad_ip.txt', 'w') as bad_ip_file:
    for bad_ip in bad_ip_list:
        bad_ip_file.write(bad_ip + '\n')

In [16]:
# Write the bad IPs to bad_ip.txt
with open('bad_ip.txt', 'w') as bad_ip_file:
    for bad_ip in bad_ip_list:
        bad_ip_file.write(bad_ip + '\n')
        
# Read the IPs back from the file
with open('bad_ip.txt', 'r') as bad_ip_file:
    ips = bad_ip_file.readlines()

# Remove duplicates by converting to a set, then back to a list
unique_ips = list(set(ips))

# Sort the list to maintain a consistent order (optional)
unique_ips.sort()

# Write the unique IPs back to the file
with open('bad_ip.txt', 'w') as bad_ip_file:
    for ip in unique_ips:
        bad_ip_file.write(ip)

In [10]:
# Read the proxy list from good-grass-proxy-list.txt
proxy_list = read_proxy_list('good-grass-proxy-list.txt')

# Get the filtered list of proxy IPs
filtered_proxies = filter_proxies(proxy_list, bad_ip_list)

In [11]:
# Write the filtered proxies to proxy-list.txt
with open('good-grass-proxy-list-filterd.txt', 'w') as proxy_file:
    for proxy in filtered_proxies:
        proxy_file.write(proxy + '\n')